In [2]:
!pip install bsedata

In [3]:
 import pandas as pd
from bsedata.bse import BSE 
import time

In [5]:
EquityDF = pd.read_excel('Datasets/Equity.xlsx', engine='openpyxl')
EquityDF .head(2)

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas


In [6]:
EquityDF .shape

(4244, 14)

𝐂𝐀𝐋𝐋𝐈𝐍𝐆 𝐁𝐒𝐄 𝐀𝐏𝐈 𝐟𝐨𝐫 𝐟𝐞𝐭𝐜𝐡𝐢𝐧𝐠 𝐒𝐭𝐨𝐜𝐤𝐬 𝐃𝐚𝐭𝐚 𝐟𝐨𝐫 𝐞𝐠: 𝐏𝐫𝐢𝐜𝐞, 𝐂𝐨𝐝𝐞, 𝐔𝐩𝐝𝐚𝐭𝐞𝐝 𝐃𝐚𝐭𝐞, 𝐎𝐩𝐞𝐧 𝐏𝐫𝐢𝐜𝐞, 𝐂𝐥𝐨𝐬𝐞 𝐏𝐫𝐢𝐜𝐞, 𝐌𝐫𝐤𝐭 𝐂𝐚𝐩

In [7]:
# Define the list of Nifty 50 stock IDs
listOf_Nifty50_StockIDs = [ 
    "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
    "UPL", "ULTRACEMCO", "WIPRO"
]

EquityDF['Security Code'] = EquityDF['Security Code'].astype(str) ##coverting into str as getQuote() accepts str input

# Filter the DataFrame to include only rows where 'Security Id' is in the list of Nifty 50 stock IDs
nifty50_OnlyDF = EquityDF[EquityDF['Security Id'].isin(listOf_Nifty50_StockIDs)].reset_index(drop=True)

# Remove spaces from column names if any
nifty50_OnlyDF.columns = nifty50_OnlyDF.columns.str.replace(' ', '')

# Display the resulting DataFrame
nifty50_OnlyDF.head(3)


,SecurityCode,IssuerName,SecurityId,SecurityName,Status,Group,FaceValue,ISINNo,Industry,Instrument,SectorName,IndustryNewName,IgroupName,ISubgroupName
0,500034,Bajaj Finance Limited,BAJFINANCE,Bajaj Finance Limited,Active,A,2.0,INE296A01024,Non Banking Financial Company (NBFC),Equity,Financial Services,Financial Services,Finance,Non Banking Financial Company (NBFC)
1,500087,"Cipla Ltd.,",CIPLA,CIPLA LTD.,Active,A,2.0,INE059A01026,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals
2,500112,"State Bank Of India,",SBIN,STATE BANK OF INDIA,Active,A,1.0,INE062A01020,Public Sector Bank,Equity,Financial Services,Financial Services,Banks,Public Sector Bank


In [8]:
nifty50_OnlyDF['SecurityCode'].values

array(['500034', '500087', '500112', '500114', '500124', '500180',
       '500182', '500209', '500228', '500247', '500300', '500312',
       '500325', '500440', '500470', '500510', '500520', '500547',
       '500570', '500696', '500790', '500800', '500820', '500825',
       '500875', '505200', '507685', '508869', '512070', '512599',
       '524715', '532174', '532187', '532215', '532281', '532454',
       '532488', '532500', '532538', '532540', '532555', '532755',
       '532898', '532921', '532977', '532978', '533278', '540005',
       '540719', '540777'], dtype=object)

In [9]:
# Initialize the BSE object, with an option to update stock codes.
bseObject = BSE(update_codes=True)

# Retrieve the 'SecurityCode' column values as a list.
sqcode_ListNifty50 = nifty50_OnlyDF['SecurityCode'].values

# Initialize an empty list to store dictionaries of stock information.
listof_stockDicts = []

# Iterate over each stock code in the Nifty 50 list.
for sqCode in sqcode_ListNifty50:
    try:
        # Fetch the stock quote for the given stock code.
        stockDict = bseObject.getQuote(sqCode)
        
        # Remove the 'buy' and 'sell' keys from the stock dictionary, if they exist.
        stockDict.pop("buy", None)
        stockDict.pop("sell", None)
        
        # Add the modified stock dictionary to the list.
        listof_stockDicts.append(stockDict)
        
        # Pause for 0.5 seconds to avoid sending too many requests in a short period.
        time.sleep(0.7)
    except IndexError:
        # If there's an error fetching data for a stock code, print an error message.
        print(f"IndexError for {sqCode}: Data not available")

# Convert the list of dictionaries into a DataFrame.
nifty50DailyTable = pd.DataFrame(listof_stockDicts)

# Display the first few rows of the DataFrame.
nifty50DailyTable.head(3)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,...,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,7122.30,233.05,3.38,19 Apr 24 | 04:00 PM,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,...,7150.60,6811.05,8190.00,5830.00,7021.46,26.80 Cr.,0.38 Lakh,0.33 Lakh,"4,40,867.75 Cr.","1,98,390.49 Cr."
1,CIPLA LTD.,1345.35,-2.30,-0.17,19 Apr 24 | 04:00 PM,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,...,1357.95,1335.35,1519.00,891.30,1345.25,4.65 Cr.,0.35 Lakh,0.33 Lakh,"1,08,619.13 Cr.","71,688.62 Cr."
2,STATE BANK OF INDIA,750.80,6.00,0.81,19 Apr 24 | 04:00 PM,SBIN,500112,A / S&P BSE SENSEX,1.00,Financial Services,...,752.25,732.05,793.50,536.90,745.00,75.91 Cr.,10.19 Lakh,7.10 Lakh,"6,70,059.86 Cr.","2,88,125.74 Cr."


In [10]:
nifty50DailyTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   companyName          50 non-null     object
 1   currentValue         50 non-null     object
 2   change               50 non-null     object
 3   pChange              50 non-null     object
 4   updatedOn            50 non-null     object
 5   securityID           50 non-null     object
 6   scripCode            50 non-null     object
 7   group                50 non-null     object
 8   faceValue            50 non-null     object
 9   industry             50 non-null     object
 10  previousClose        50 non-null     object
 11  previousOpen         50 non-null     object
 12  dayHigh              50 non-null     object
 13  dayLow               50 non-null     object
 14  52weekHigh           50 non-null     object
 15  52weekLow            50 non-null     object
 16  weightedAv

In [11]:
nifty50DailyTable.shape

(50, 22)

In [12]:
# Rename columns for clarity
nifty50DailyTable.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50DailyTable.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)

# Convert 'updatedOn' column to datetime and extract date
nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check for any invalid or missing date values
if pd.isna(nifty50DailyTable['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'].dt.date)

# Check if 'totalTradedValueCr' column exists, if not, create it and convert related columns to numeric
if 'totalTradedValueCr' not in nifty50DailyTable.columns:
    nifty50DailyTable['totalTradedValueCr'] = pd.to_numeric(nifty50DailyTable['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')
    nifty50DailyTable['totalTradedQuantityLakh'] = pd.to_numeric(nifty50DailyTable['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')
    nifty50DailyTable['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50DailyTable['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')
    nifty50DailyTable['marketCapFullCr'] = pd.to_numeric(nifty50DailyTable['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')
    nifty50DailyTable['marketCapFreeFloatCr'] = pd.to_numeric(nifty50DailyTable['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')

    # Drop original columns
    nifty50DailyTable.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

# Display the first 2 rows of the DataFrame
nifty50DailyTable.head(n=2)


,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,...,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,7122.30,233.05,3.38,2024-04-19,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,...,7150.60,6811.05,8190.00,5830.00,7021.46,26.80,0.38,0.33,440867.75,198390.49
1,CIPLA LTD.,1345.35,-2.30,-0.17,2024-04-19,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,...,1357.95,1335.35,1519.00,891.30,1345.25,4.65,0.35,0.33,108619.13,71688.62


In [11]:
nifty50DailyTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   companyName              47 non-null     object        
 1   currentValue             47 non-null     object        
 2   change                   47 non-null     object        
 3   pChange                  47 non-null     object        
 4   updatedOn                47 non-null     datetime64[ns]
 5   securityID               47 non-null     object        
 6   scripCode                47 non-null     object        
 7   sharegroup               47 non-null     object        
 8   faceValue                47 non-null     object        
 9   industry                 47 non-null     object        
 10  previousClose            47 non-null     object        
 11  previousOpen             47 non-null     object        
 12  dayHigh                  47 non-null  

In [12]:
import sqlite3
import pandas as pd

database_path = 'nifty50.db'

# Connect to the SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Function to create a table if it doesn't exist
def create_table_if_not_exists(table_name, schema):
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    if cursor.fetchone() is None:
        cursor.execute(schema)
        conn.commit()

# Creating the nifty50_dailydata table if it doesn't exist
nifty50_dailydata_schema = '''
CREATE TABLE nifty50_dailydata (
    companyName TEXT,
    currentValue REAL,
    change REAL,
    pChange REAL,
    updatedOn DATE,
    securityID TEXT,
    scripCode TEXT,
    sharegroup TEXT,
    faceValue REAL,
    industry TEXT,
    previousClose REAL,
    previousOpen REAL,
    dayHigh REAL,
    dayLow REAL,
    fiftytwoweekHigh REAL,
    fiftytwoweekLow REAL,
    weightedAvgPrice REAL,
    totalTradedQuantityLakh REAL,
    totalTradedValueCr REAL,
    twoWeekAvgQuantityLakh REAL,
    marketCapFullCr REAL,
    marketCapFreeFloatCr REAL
);
'''
create_table_if_not_exists('nifty50_dailydata', nifty50_dailydata_schema)

# Checking for the latest date in the nifty50_dailydata table
cursor.execute('SELECT MAX(updatedOn) FROM nifty50_dailydata;')
sql_max_updatedOn = cursor.fetchone()[0]

# Assuming nifty50DailyTable is a pandas DataFrame with your data
df_max_updatedOn = nifty50DailyTable['updatedOn'].max()

if sql_max_updatedOn is None or (pd.to_datetime(df_max_updatedOn) > pd.to_datetime(sql_max_updatedOn)):
    nifty50DailyTable.to_sql('nifty50_dailydata', conn, if_exists='append', index=False)
    print("Data inserted or updated successfully.")
else:
    print("No new data to append.")

# Creating the nifty50_companydata table if it doesn't exist
nifty50_companydata_schema = '''
CREATE TABLE nifty50_companydata (
    securityCode TEXT,
    issuerName TEXT,
    securityId TEXT,
    securityName TEXT,
    status TEXT,
    CompanyGroup TEXT,
    faceValue REAL,
    isinNo TEXT,
    industry TEXT,
    instrument TEXT,
    sectorName TEXT,
    industryNewName TEXT,
    igroupName TEXT,
    iSubgroupName TEXT
);
'''
create_table_if_not_exists('nifty50_companydata', nifty50_companydata_schema)

# Close the database connection
conn.close()


Data inserted or updated successfully.
